In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

## Store Part I results into DataFrame

In [2]:
#Load the csv exported in Part I to a DataFrame
csv_path = ("Resources/output_data_file")
weather_df = pd.read_csv(csv_path)

weather_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Labuhan,55,ID,1588973120,89,-6.88,112.21,79.14,6.53
1,1,Buraidah,75,SA,1588973121,29,26.33,43.98,87.80,24.16
2,2,Morant Bay,20,JM,1588973121,74,17.88,-76.41,86.00,28.86
3,3,Dakar,3,SN,1588972978,88,14.69,-17.44,71.60,18.34
4,4,Ostrovnoy,99,RU,1588973122,93,68.05,39.51,33.28,4.74
...,...,...,...,...,...,...,...,...,...,...
553,553,Karratha,54,AU,1588973267,47,-20.74,116.85,76.21,8.01
554,554,Madona,17,LV,1588973267,90,56.85,26.22,42.26,5.79
555,555,Koutsouras,0,GR,1588973267,68,35.03,25.95,62.64,19.62
556,556,Sibolga,98,ID,1588973173,82,1.74,98.78,78.44,2.26


## Humidity Heatmap

In [30]:
#Configure gmaps.
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"]

In [34]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

## Create New Data Frame fitting weather criteria

In [111]:
#Narrow down the cities to fit weather conditions.
No_cloud = weather_df[weather_df["Cloudiness"] == 0]
Temp_above_70 = No_cloud[No_cloud["Max Temp"] > 70]
Temp_below_85 = Temp_above_70[Temp_above_70["Max Temp"] < 85]
Wind_below_15 = Temp_below_85[Temp_below_85["Wind Speed"] < 15]
Perfect_weather = Wind_below_15[Wind_below_15["Humidity"] < 45]
#Set Index
indexed_Perfect_weather = Perfect_weather.reset_index()
del indexed_Perfect_weather["index"]
indexed_Perfect_weather.count()

Unnamed: 0    8
City          8
Cloudiness    8
Country       7
Date          8
Humidity      8
Lat           8
Lng           8
Max Temp      8
Wind Speed    8
dtype: int64

In [112]:
#Drop any rows will null values.
indexed_Perfect_weather= indexed_Perfect_weather.dropna(how='any')

In [113]:
indexed_Perfect_weather.count()

Unnamed: 0    7
City          7
Cloudiness    7
Country       7
Date          7
Humidity      7
Lat           7
Lng           7
Max Temp      7
Wind Speed    7
dtype: int64

In [114]:
indexed_Perfect_weather

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,49,Tura,0,IN,1588973001,43,25.52,90.22,72.61,3.56
1,186,Jalu,0,LY,1588973070,24,29.03,21.55,70.74,14.27
2,218,Jashpurnagar,0,IN,1588973180,24,22.90,84.15,76.66,7.70
3,322,Kānker,0,IN,1588973208,36,20.27,81.49,83.79,5.75
4,359,Guerrero Negro,0,MX,1588973217,36,27.98,-114.06,81.59,13.06
6,505,Safaga,0,EG,1588973254,36,26.73,33.94,77.00,4.38
7,525,Monteagudo,0,BO,1588973260,44,-19.82,-63.98,70.70,4.43


## Hotel Map

In [140]:
#Store into variable named hotel_df.
hotel_df = []
#Add a "Hotel Name" column to the DataFrame.
indexed_perfect_vacation["Hotel Name"]=""
indexed_perfect_vacation
#params dictionary to update each iteration
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


# Use the lat/lng we recovered to identify Hotels
for index, row in indexed_perfect_vacation.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    # make request and print url
    response = requests.get(base_url, params=params).json()
    
     # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        indexed_perfect_vacation.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    

Retrieving Results for Index 0: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
------------
Retrieving Results for Index 1: Jalu.
Closest hotel in Jalu is Jalu Hotel.
------------
Retrieving Results for Index 2: Jashpurnagar.
Closest hotel in Jashpurnagar is Hotel Nirvana, Jashpur.
------------
Retrieving Results for Index 3: Kānker.
Closest hotel in Kānker is Kanker Palace Heritage.
------------
Retrieving Results for Index 4: Guerrero Negro.
Closest hotel in Guerrero Negro is Hotel Los Caracoles.
------------
Retrieving Results for Index 5: Walvis Bay.
Closest hotel in Walvis Bay is Protea Hotel by Marriott Walvis Bay Pelican Bay.
------------
Retrieving Results for Index 6: Safaga.
Closest hotel in Safaga is lagoonie Lodge Hotel.
------------
Retrieving Results for Index 7: Monteagudo.
Closest hotel in Monteagudo is Hotel Fortin.
------------


In [135]:
indexed_perfect_vacation

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,49,Tura,0,IN,1588973001,43,25.52,90.22,72.61,3.56,Hotel Polo Orchid
1,186,Jalu,0,LY,1588973070,24,29.03,21.55,70.74,14.27,Jalu Hotel
2,218,Jashpurnagar,0,IN,1588973180,24,22.90,84.15,76.66,7.70,"Hotel Nirvana, Jashpur"
3,322,Kānker,0,IN,1588973208,36,20.27,81.49,83.79,5.75,Kanker Palace Heritage
4,359,Guerrero Negro,0,MX,1588973217,36,27.98,-114.06,81.59,13.06,Hotel Los Caracoles
5,450,Walvis Bay,0,NaN,1588973241,37,-22.96,14.51,73.40,2.84,Protea Hotel by Marriott Walvis Bay Pelican Bay
6,505,Safaga,0,EG,1588973254,36,26.73,33.94,77.00,4.38,lagoonie Lodge Hotel
7,525,Monteagudo,0,BO,1588973260,44,-19.82,-63.98,70.70,4.43,Hotel Fortin


In [136]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Lat", "Lng"]] 

In [138]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))